In [2]:
import ndjson
import numpy as np

path = '/home/yigit/phd/data/trajnetplusplusdataset/output/train/lcas.ndjson'

with open(path) as f:
    data = ndjson.load(f)

def get_person_track(pid):
    x, y = [], []
    for line in data:
        try:
            track = line['track']
            if int(track['p']) == pid:
                x.append(track['x'])
                y.append(track['y'])
        except Exception as e:
            pass
    return np.array(x).reshape(-1, 1), np.array(y).reshape(-1, 1)


FileNotFoundError: [Errno 2] No such file or directory: '/home/yigit/phd/data/trajnetplusplusdataset/output/train/lcas.ndjson'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 8]

xs, ys = [], []
num_people = 0

for pid in range(588,941):
    x, y = get_person_track(pid)
    if len(x) < 5:  # choose paths longer than 5 steps
        continue
    else:
        xs.append(x)
        ys.append(y)
        num_people += 1

In [ ]:
for i in range(num_people):
    x, y = xs[i], ys[i]
    plt.plot(x, y)
plt.title('All Paths')
plt.xlabel('x position')
plt.ylabel('y position')
plt.show()

In [ ]:
ee_xs, ee_ys = [], []

for i in range(num_people):
    x, y = xs[i], ys[i]
    eligible = False
    for step in range(len(x)):
        # a set of paths with the following properties seem eligible as demonstrations
        if -25 < x[step] < -10 and -5 <= y[step] <= -0:
            for step2 in range(len(x)):
                if 10 < x[step2] < 20:
                    eligible = True
                    break
    if eligible:
        plt.plot(x, y, label=str(i))
        ee_xs.append(x)
        ee_ys.append(y)
plt.plot([0], [0], 'ko')
plt.title('Demonstrations')
plt.xlabel('x position')
plt.ylabel('y position')
plt.legend()
plt.show()

#### Filtering out some

In [ ]:
e_xs, e_ys = [], []
for i in range(len(ee_xs)):
    eligible = True
    x, y = ee_xs[i], ee_ys[i]
    for step in range(len(x)):
        if (10 < x[step] < 20) and ((y[step] <= -1.5) or (y[step] > 4)):
            eligible = False
            break
    if eligible:
        plt.plot(x, y, label=str(i))
        e_xs.append(x)
        e_ys.append(y)
plt.plot([0], [0], 'ko')
plt.title('Demonstrations')
plt.xlabel('x position')
plt.ylabel('y position')
plt.legend()
plt.show()

In [ ]:
# normalizing demonstration lengths
# first clipping along x

# max of min_x's and min of max_x's are the limits
minx, maxx = -1000000, 1000000
for i in range(len(e_xs)):
    cur_min = np.min(e_xs[i])
    cur_max = np.max(e_xs[i])
    minx = cur_min if cur_min > minx else minx
    maxx = cur_max if cur_max < maxx else maxx

for i in range(len(e_xs)):
    clipped_indices = np.where(np.logical_and((minx <= e_xs[i]), (e_xs[i] <= maxx)))
    e_xs[i] = e_xs[i][clipped_indices]
    e_ys[i] = e_ys[i][clipped_indices]

In [ ]:
from scipy import interpolate

xs = np.linspace(start=minx, stop=maxx, num=200)
d_xs, d_ys = [], []

for i in range(len(e_xs)):
    f = interpolate.interp1d(e_xs[i], e_ys[i], kind='cubic', fill_value="extrapolate")
    d_ys.append(f(xs))
    
xs = ((xs) / (maxx - minx)) * 2  # normalizing x vals

for i in range(len(d_ys)):
    d_xs.append(xs)
    plt.plot(d_xs[i], d_ys[i], label=str(i))

plt.plot([0], [0], 'ko', label = 'robot')  # robot as a point
plt.title('Demonstrations')
plt.xlabel('x position')
plt.ylabel('y position')
plt.text(-1, 2.1, 'path length: '+str(200))
plt.legend(loc='lower right')
plt.show()

#### Info on data

In [ ]:
print(f'x_demonstrations_shape={np.array(d_xs).shape}')
print(f'y_demonstrations_shape={np.array(d_ys).shape}')

In [ ]:
data_path = '/home/yigit/phd/yigit_phd_thesis/cnmp/data/'

# reshaping to meet CNMP requirements
d_xs = np.array(d_xs).reshape((np.shape(d_xs)[0], np.shape(d_xs)[1], 1))
d_ys = np.array(d_ys).reshape((np.shape(d_ys)[0], np.shape(d_ys)[1], 1))

# saving last 2 for validation and the rest for training
np.save(data_path+'d_x.npy', np.array(d_xs[:-2]))
np.save(data_path+'d_y.npy', np.array(d_ys[:-2]))
np.save(data_path+'v_d_x.npy', np.array(d_xs[-2:]))
np.save(data_path+'v_d_y.npy', np.array(d_ys[-2:]))